<div style="text-align: right">INFO 6105 Data Science Eng Methods and Tools, Lecture 9 Day 1</div>
<div style="text-align: right">Dino Konstantopoulos, 31 October 2022, with material by by John K. Kruschke</div>

# Bayesian Hierarchies
Just like **objects** derive from other **objects** in OO programming, ***specialized*** models derive from more ***abstract*** models in Data Science. In this way, we can produce hierarchies that yield more abstract models from more specialized datasets. 

<br />
<center>
<img src="ipynb.images/phillies-astros.jfif" width=600 />
</center>

Every summer, americans get together to Watch what they call the ***World Series*** of baseball.

This summer, it's the Philadelphia Phillies versus the Houston Astros.

We've modelled drivers and teams, but how do we model ***groups*** of teams or players?

For example, all greek soccer teams versus all English soccer teams? I bet the english soccer teams come on top.

We model that with Bayesian **hierarchies**.

The hierarchical structure of a model is an expression of how you think the data should be meaningfully modeled and the model description captures *group* aspects of the data that you care about. Using Bayesian estimation, you supply a prior distribution on group parameters, and infer an entire posterior distributions across the joint parameter space.

In World Series, pitchers get to bat as well. But pitchers are traine to *throw* a ball, not to *bat* it. Is that the same in cricket? I bet that pitchers have much worse batting averages than batters, but how to prive that with Bayesian simulations?

## Batting avergages in Baseball
During a year of games, different players have different numbers of opportunities at bat, and on some of these opportunities a player might actually hit the ball.

That ratio, of hits divided by opportunities at bat, is called the [batting average](https://en.wikipedia.org/wiki/Batting_average_(baseball)) of each player.

Assume we have data consisting of records from 948 players in the 2012 regular season of Major League Baseball.

To give some sense of the data, there were 324 pitchers with a median of 4.0 at bats, 103 catchers with a median of 170.0 at - bats, and 60 right fielders with a median of 340.5 at bats, along with 461 players in six other positions.

For every pair of players, we could ask how much their estimated batting abilities differ. 

For every pair of positions, we can also ask how much their batting abilities differ. For example, do outfielders have different batting averages than basemen?

We need to estimate the batting abilities for individual players, for positions, and for groups of players. Clearly, we expect the  batting ability of pitchers to be lower than that for catchers, for example.

Each category has its own modal bias $ω_c$, from which all subject biases in the category are assumed to be drawn.

When $κ$ is large, the category biases $ω_c$ are tightly concentrated.

A prior on $κ_c$ applies independently to each $κ_c$ in a manner fixed by the prior constants $\alpha_κ$ and $beta_κ$, and the $κ_c$’s do not mutually inform each other via that part of the hierarchy.

Each row could also contain a unique subject identifier.

Let's start by putting all 948 players under a single over-arching distribution (the pooled model), then the estimates for two players with identical batting records would be identical regardless of the position they play.

Players with many at bats should have somewhat less shrinkage of their individual estimates than players with few at-bats and who maybe had estimates dominated by position information.

In [49]:
import numpy as np
np.random.seed(0) # to keep it reproducible
import pymc3 as pm
import arviz as az
import pandas as pd

In [50]:
df = pd.concat(
    map(pd.read_csv, ['data/BattingAverage.csv']), ignore_index=True)
df.head()

,Player,PriPos,Hits,AtBats,PlayerNumber,PriPosNumber
0,Fernando Abad,Pitcher,1,7,1,1
1,Bobby Abreu,Left Field,53,219,2,7
2,Tony Abreu,2nd Base,18,70,3,4
3,Dustin Ackley,2nd Base,137,607,4,4
4,Matt Adams,1st Base,21,86,5,3


In [51]:
num_players = len(df)
num_players

948

Let's explain the intuition behind the parametric form of the [beta distribution](https://en.wikipedia.org/wiki/Beta_distribution) a bit more. Very important for baseball. 

<br />
<center>
<img src="ipynb.images/atbat.png" width=400 />
</center>

Whereas the Poisson distribution is a *count* statistic, the Beta distribution can be thought of as representing a distribution of probabilities- that is, it represents all the possible values of a probability when we don't know what that probability is. The domain of the Beta distribution is (0, 1), just like a probability, so we already know we're on the right track.

Imagine you want to predict a baseball player's season-long batting average.  We know it's somewhere around .300, like we know the average number of text phones I can expect my girlfriend to receive per day before I get worried too much.

The baseball player can get into a lucky streak and get an average of 1.000, or an unlucky streak with an average of 0, neither of which are a good predictor. Let's go in with prior expectations: In history, most season batting averages have hovered between something like .200 and .350.

Let's plot a Beta distribution with parameters α=81 and β=219. We know that the formula for the mean of the beta distribution is:

$$ \frac{\alpha}{\alpha + \beta} = \frac{81}{81+219} =.270$$

and the distribution lies almost entirely within the reasonable range for a batting average \[.2, .35\].

Plot histogram:
```
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

data = stats.beta.rvs(81, 219, loc = 0, scale = 1, size=1000)
plt.hist(data,bins='auto',normed=True)
```

Imagine now the player gets his first at bat and has a single hit. His record for the season is now 1 hit; 1 at bat. 

We update our probabilities by shifting the entire curve over just a bit to reflect our new information. The new Beta distribution will be:

$$ \text{Beta}(\alpha_0 + \text{hits}, \beta_0 + \text{misses})$$
 
Where $α_0$ and $β_0$ are the parameters we started with: 81 and 219. Thus, in this case, $α$ has increased by 1 (his one hit), while $β$ has not increased at all (no misses yet). Read [here](https://en.wikipedia.org/wiki/Conjugate_prior#Example) for proof. It's about *conjugate priors*: The beta distribution is a conjugate prior for a binomial data likelihood. This means that if the data likelihood is a binomial and the prior is a beta, the posterior will also be a beta.

Suppose halfway through the season he has been up to bat 300 times, hitting 150 out of those times. Let's plot!
```
data = stats.beta.rvs(81 + 150, 219 + 150, loc = 0, scale = 1, size=1000)
plt.hist(data,bins='auto',normed=True)
```

Notice the curve is now both *thinner* (0.44 - 0.34 = 0.10 versus 0.34 - 0.20 = 0.14) and *shifted* to the right (higher batting average) than it used to be: We have a better sense of what the player's batting average is!

The Beta distribution is best for representing a probabilistic distribution of probabilities- the case where we don't know what a probability is in advance, but we have reasonable guesses.

# Reparametrization of the Beta
We reparametrize $\alpha$ and $\beta$ into $\omega$ and $\kappa$:
```
omega = 2
kappa = 1
alpha, beta = omega*(kappa - 1) + 1, (1 - omega)*(kappa - 2) + 1
alpha, beta
```

```
# prior parameters
omega = 0.5
kappa = 100
alpha, beta = omega*(kappa - 1) + 1, (1 - omega)*(kappa - 2) + 1

# calculate posterior distribution, using a beta distribution
variates = stats.beta(alpha, beta)

# beta distribution
xs = np.linspace(0, 1, num=1000)
pdf = variates.pdf(xs)
plt.plot(pdf)
plt.xlabel('x')
plt.ylabel('y')
plt.title('pdf')
plt.grid(True)
plt.show()
```

We see that for high values of kappa, we have high certainty.

In [ ]:
# prior parameters
omega = 0.5
kappa = 10
alpha, beta = omega*(kappa - 1) + 1, (1 - omega)*(kappa - 2) + 1

# calculate posterior distribution, using a beta distribution
variates = stats.beta(alpha, beta)

# beta distribution
xs = np.linspace(0, 1, num=1000)
pdf = variates.pdf(xs)
plt.plot(pdf)
plt.xlabel('x')
plt.ylabel('y')
plt.title('pdf')
plt.grid(True)
plt.show()

For lower values of kappa, not so sure anymore.

In [ ]:
# prior parameters
omega = 0.5
kappa = 4
alpha, beta = omega*(kappa - 1) + 1, (1 - omega)*(kappa - 2) + 1

# calculate posterior distribution, using a beta distribution
variates = stats.beta(alpha, beta)

# beta distribution
xs = np.linspace(0, 1, num=1000)
pdf = variates.pdf(xs)
plt.plot(pdf)
plt.xlabel('x')
plt.ylabel('y')
plt.title('pdf')
plt.grid(True)
plt.show()

Then, for even lower values of kappa, strange things happen: We get a bias in one direction and we observe more and more one-sided results, eventually down to an absolute certainty!

In [ ]:
# prior parameters
omega = 0.5
kappa = 3
alpha, beta = omega*(kappa - 1) + 1, (1 - omega)*(kappa - 2) + 1

# calculate posterior distribution, using a beta distribution
variates = stats.beta(alpha, beta)

# beta distribution
xs = np.linspace(0, 1, num=1000)
pdf = variates.pdf(xs)
plt.plot(pdf)
plt.xlabel('x')
plt.ylabel('y')
plt.title('pdf')
plt.grid(True)
plt.show()

In [ ]:
# prior parameters
omega = 0.5
kappa = 2.5
alpha, beta = omega*(kappa - 1) + 1, (1 - omega)*(kappa - 2) + 1

# calculate posterior distribution, using a beta distribution
variates = stats.beta(alpha, beta)

# beta distribution
xs = np.linspace(0, 1, num=1000)
pdf = variates.pdf(xs)
plt.plot(pdf)
plt.xlabel('x')
plt.ylabel('y')
plt.title('pdf')
plt.grid(True)
plt.show()

In [ ]:
# prior parameters
omega = 0.5
kappa = 2.05
alpha, beta = omega*(kappa - 1) + 1, (1 - omega)*(kappa - 2) + 1

# calculate posterior distribution, using a beta distribution
variates = stats.beta(alpha, beta)

# beta distribution
xs = np.linspace(0, 1, num=1000)
pdf = variates.pdf(xs)
plt.plot(pdf)
plt.xlabel('x')
plt.ylabel('y')
plt.title('pdf')
plt.grid(True)
plt.show()

In [ ]:
# prior parameters
omega = 0.5
kappa = 2
alpha, beta = omega*(kappa - 1) + 1, (1 - omega)*(kappa - 2) + 1

# calculate posterior distribution, using a beta distribution
variates = stats.beta(alpha, beta)

# beta distribution
xs = np.linspace(0, 1, num=1000)
pdf = variates.pdf(xs)
plt.plot(pdf)
plt.xlabel('x')
plt.ylabel('y')
plt.title('pdf')
plt.grid(True)
plt.show()

In [ ]:
# prior parameters
omega = 0.5
kappa = 1.9
alpha, beta = omega*(kappa - 1) + 1, (1 - omega)*(kappa - 2) + 1

# calculate posterior distribution, using a beta distribution
variates = stats.beta(alpha, beta)

# beta distribution
xs = np.linspace(0, 1, num=1000)
pdf = variates.pdf(xs)
plt.plot(pdf)
plt.xlabel('x')
plt.ylabel('y')
plt.title('pdf')
plt.grid(True)
plt.show()

In [ ]:
# prior parameters
omega = 0.5
kappa = 1.5
alpha, beta = omega*(kappa - 1) + 1, (1 - omega)*(kappa - 2) + 1

# calculate posterior distribution, using a beta distribution
variates = stats.beta(alpha, beta)

# beta distribution
xs = np.linspace(0, 1, num=1000)
pdf = variates.pdf(xs)
plt.plot(pdf)
plt.xlabel('x')
plt.ylabel('y')
plt.title('pdf')
plt.grid(True)
plt.show()

In [ ]:
# prior parameters
omega = 0.5
kappa = 1
alpha, beta = omega*(kappa - 1) + 1, (1 - omega)*(kappa - 2) + 1

# calculate posterior distribution, using a beta distribution
variates = stats.beta(alpha, beta)

# beta distribution
xs = np.linspace(0, 1, num=1000)
pdf = variates.pdf(xs)
plt.plot(pdf)
plt.xlabel('x')
plt.ylabel('y')
plt.title('pdf')
plt.grid(True)
plt.show()

In [ ]:
# prior parameters
omega = 0.5
kappa = 0.01
alpha, beta = omega*(kappa - 1) + 1, (1 - omega)*(kappa - 2) + 1

# calculate posterior distribution, using a beta distribution
variates = stats.beta(alpha, beta)

# beta distribution
xs = np.linspace(0, 1, num=1000)
pdf = variates.pdf(xs)
plt.plot(pdf)
plt.xlabel('x')
plt.ylabel('y')
plt.title('pdf')
plt.grid(True)
plt.show()

Interesting reparametrization!

# The Single hierarchy Model
We assume that each player has a different hitting average, but we shrink that to a league-average model.

Since we have a count of `Hits`` and a number of `AtBats`, the **binomial** is our optimal data likelihood pdf for our dataset.

We use the same omega/kappa decomposition we used above:
```
with pm.Model() as single_hierarchy_model:
    # hyperpriors
    omega = pm.Beta('omega', 1., 1.)
    kappa_minus2 = pm.Gamma('kappa_minus2', 0.01, 0.01)
    kappa = pm.Deterministic('kappa', kappa_minus2 + 2)
    
    theta = pm.Beta('theta', alpha=omega*(kappa-2)+1, beta=(1-omega)*(kappa-2)+1, shape=num_players)
    y = pm.Binomial('y', n=df['AtBats'], p=theta, observed=df['Hits'])
```

We run:
```
with single_hierarchy_model:
    single_hierarchy_trace = pm.sample(cores=1)
```

Plotting [energy transition distribution and marginal energy distribution](https://arviz-devs.github.io/arviz/api/generated/arviz.plot_energy.html) in Hamiltonian Monte Carlo algorithms may help diagnose poor exploration of state space:
```
az.plot_energy(single_hierarchy_trace)
```

```
pm.model_to_graphviz(single_hierarchy_model)
```

Here's a way to compare all players with each other:
```
az.plot_forest(single_hierarchy_trace, var_names=['theta'], combined=True)
```

# Double Hierarchy Model
Now, we assume that positions are relevant, and that each position has distinct underlying statistics that every player inherits from.
```
num_positions = df['PriPosNumber'].nunique()
position_idx = df['PriPosNumber']
num_players = df['PlayerNumber'].nunique()
player_idx = df['PlayerNumber']

position_idx
```

There are 9 different positions in baseball:

In [55]:
num_positions

9

In [56]:
set(position_idx.values)

{1, 2, 3, 4, 5, 6, 7, 8, 9}

Let's give them an index:

In [57]:
position_idx2 = df['PriPosNumber'] - 1
position_idx2

0      0
1      6
2      3
3      3
4      2
      ..
943    0
944    4
945    0
946    0
947    8
Name: PriPosNumber, Length: 948, dtype: int64

In [58]:
set(position_idx2.values)

{0, 1, 2, 3, 4, 5, 6, 7, 8}

Let's set up our simulation:
```
with pm.Model() as double_hierarchy_model:
    # hyper-hyperpriors
    omega = pm.Beta('omega', 1., 1.)
    kappa_minus2 = pm.Gamma('kappa_minus2', 0.01, 0.01)
    kappa = pm.Deterministic('kappa', kappa_minus2 + 2)

    # Parameters for positions (hyperpriors)
    omega_c = pm.Beta('omega_c',
                       omega*(kappa-2)+1, (1-omega)*(kappa-2)+1,
                       shape = num_positions)    
    kappa_c_minus2 = pm.Gamma('kappa_c_minus2',
                              0.01, 0.01,
                              shape = num_positions)
    kappa_c = pm.Deterministic('kappa_c', kappa_c_minus2 + 2)
    
    # Parameters for players (priors)
    #theta = pm.Beta('theta', alpha=omega*(kappa-2)+1, beta=(1-omega)*(kappa-2)+1, shape=num_players)
    theta = pm.Beta('theta',
                     omega_c[position_idx2]*(kappa_c[position_idx2]-2)+1,
                    (1-omega_c[position_idx2])*(kappa_c[position_idx2]-2)+1,
                     shape = num_players)
    
    y = pm.Binomial('y', n=df['AtBats'], p=theta, observed=df['Hits'])
```

Let's visualize:
```
pm.model_to_graphviz(double_hierarchy_model)
```

Let's simulate:
```
with double_hierarchy_model:
    double_hierarchy_trace = pm.sample(cores=1)
```

Let's plot the energy of the Hamiltonian to see if we converged:
```
az.plot_energy(double_hierarchy_trace)
```

Hmmm..... Let's rerun with more iterations:
```
with double_hierarchy_model:
    double_hierarchy_trace2 = pm.sample(2000, cores=1)
```

```
az.plot_energy(double_hierarchy_trace2)
```

Not much improvement :-(

Forest plot:
```
az.plot_forest(double_hierarchy_trace, var_names=['omega_c'], combined=True)
```

And now we can confirm that pitchers ***suck*** big-time compared to batters!

<br />
<center>
<img src="ipynb.images/minions-laughing.gif" width=500 />
</center>

Higher values of kappa imply that batting averages are more concentrated for the category, whereas lower values imply bigger deviations and the HDI also showcases uncertainties. 
```
az.plot_forest(double_hierarchy_trace, var_names=['kappa_c'], combined=True)
```

# Homework (optional)
Can you think of another data model that you can introduce hierarchies in order to simulate less-derived categories?

# Reference

The [puppy dog book](https://www.amazon.com/Doing-Bayesian-Data-Analysis-Tutorial/dp/0124058884)!

[Doing Bayesian Data Analysis, Second Edition:
A Tutorial with R, JAGS, and Stan](https://jkkweb.sitehost.iu.edu/DoingBayesianDataAnalysis/)